In [1]:
import numpy as np
import pandas as pd
import time
import skimage.measure
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from matplotlib import animation
import matplotlib.patches as mpatches
from numpy import diff

#Transpose of 3x3 matrix
def TP(x): 
    a11 = x[0,0]
    a12 = x[0,1]
    a13 = x[0,2]
    a21 = x[1,0]
    a22 = x[1,1]
    a23 = x[1,2]
    a31 = x[2,0]
    a32 = x[2,1]
    a33 = x[2,2]
    x = np.array([[a11,a21,a31],[a12,a22,a32],[a13,a23,a33]])
    return x
#multiply of 3x3 matrices
def DT(x,y):
    a11 = x[0,0]*y[0,0]+x[0,1]*y[1,0]+x[0,2]*y[2,0]
    a12 = x[0,0]*y[0,1]+x[0,1]*y[1,1]+x[0,2]*y[2,1]
    a13 = x[0,0]*y[0,2]+x[0,1]*y[1,2]+x[0,2]*y[2,2]
    a21 = x[1,0]*y[0,0]+x[1,1]*y[1,0]+x[1,2]*y[2,0]
    a22 = x[1,0]*y[0,1]+x[1,1]*y[1,1]+x[1,2]*y[2,1]
    a23 = x[1,0]*y[0,2]+x[1,1]*y[1,2]+x[1,2]*y[2,2]
    a31 = x[2,0]*y[0,0]+x[2,1]*y[1,0]+x[2,2]*y[2,0]
    a32 = x[2,0]*y[0,1]+x[2,1]*y[1,1]+x[2,2]*y[2,1]
    a33 = x[2,0]*y[0,2]+x[2,1]*y[1,2]+x[2,2]*y[2,2]
    x = np.array([[a11,a21,a31],[a12,a22,a32],[a13,a23,a33]])
    return x
#trace of 3x3 matrix(AT A)
def TRA(x):
    tra = (x[0,0]*x[1,1]*x[2,2])
    return tra

i = 0 # i1 for T, i2 for U, I3 for V, i4 for W, i5 for all data
Start = 250 # numer of rows in dataset
End = 260# time start

sk = Start*45*45*700
nr = End - Start
n = 45*45*700*nr
nx = 45
ny = 45
nz = 700
dx = 400
print(nr)

#Importing data

tic = time.perf_counter()
# dft = pd.read_csv('/Users/thanhtran/Dr.Bh_lab/Helium_FLUX_1/He_h01_01_T.csv',skiprows = sk,nrows = n, dtype =np.float32)
# datat = dft.to_numpy()
dfu = pd.read_csv('C:/Users/Sound of Silence/Desktop/Helium_FLUX_1/He_h01_01_U.csv',skiprows = sk,nrows = n, header = None, dtype =np.float32)#header = None)
datau = dfu.to_numpy()
print(datau.shape)
del dfu
dfv = pd.read_csv('C:/Users/Sound of Silence/Desktop/Helium_FLUX_1/He_h01_01_V.csv',skiprows = sk,nrows = n, header = None, dtype =np.float32)#header = None)
datav = dfv.to_numpy()
del dfv
dfw = pd.read_csv('C:/Users/Sound of Silence/Desktop/Helium_FLUX_1/He_h01_01_W.csv',skiprows = sk,nrows = n, header = None, dtype =np.float32)#header = None)
dataw = dfw.to_numpy()
del dfw
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

#Shaping data into actural shape (45*45*700)
# datat = np.reshape(datat, (nx,ny,nz,nr), order="F")
datau = np.reshape(datau, (nx,ny,nz,nr), order="F")
datav = np.reshape(datav, (nx,ny,nz,nr), order="F")
dataw = np.reshape(dataw, (nx,ny,nz,nr), order="F")

10
(14175000, 1)
Downloaded the tutorial in 34.7541 seconds


In [3]:
dx = dy = 40
dz = 10

import fluidlab as fl
dataud,datavd,datawd = fl.uvw_wcdless(datau,datav,dataw)
tic = time.perf_counter()
omega=np.zeros((nx,ny,nz,nr),dtype=np.float32)
for t in range(0,nr):
    gu = np.gradient(datau[:,:,:,t],dx,dy,dz)
    gv = np.gradient(datav[:,:,:,t],dx,dy,dz)
    gw = np.gradient(dataw[:,:,:,t],dx,dy,dz)
    
    GV = np.array([[gu[0],gv[0],gw[0]],[gu[1],gv[1],gw[1]],[gu[2],gv[2],gw[2]]])
    GVT = TP(GV)

    A = 0.5*(GV+GVT)
    B = 0.5*(GV-GVT)
    AT = TP(A)
    BT = TP(B)
    ATA = DT(AT,A)
    BTB = DT(BT,B)
    a = TRA(ATA)
    b = TRA(BTB)
    ef=0.001*np.max(b-a)
    omega[:,:,:,t] = b/(a+b+ef)
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

Downloaded the tutorial in 6.9667 seconds


In [4]:
omegafl2 = fl.omega(datau,datav,dataw,dx=40,dy=40,dz=10)

In [10]:
lamb2 = fl.lambda2(datau,datav,dataw,dx=40,dy=40,dz=10)

In [11]:
tic = time.perf_counter()
#Saving File

filename = "lambda2.csv"
filedata = lamb2

import csv
nx,ny,nz,nt = filedata.shape[0],filedata.shape[1],filedata.shape[2],filedata.shape[3]
filedata = np.reshape(filedata, (nx*ny*nz*nt,1), order="F")
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile) 
    # writing the data rows   
    csvwriter.writerows(filedata)
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

Downloaded the tutorial in 35.9624 seconds


In [6]:
omega[22,:,:,1]

array([[2.15348586e-01, 3.09194606e-02, 9.46584903e-03, ...,
        1.47321198e-18, 1.12621203e-17, 3.92877621e-18],
       [4.79610683e-03, 2.07138900e-02, 4.58667614e-02, ...,
        4.03038559e-17, 1.84101933e-17, 1.03109621e-19],
       [9.81253479e-03, 3.01088579e-02, 3.81230097e-03, ...,
        1.33203776e-19, 3.30984452e-18, 4.74768502e-18],
       ...,
       [9.35612619e-02, 4.52785268e-02, 9.06258501e-05, ...,
        3.37361543e-17, 2.29487067e-17, 2.75891022e-17],
       [4.93194461e-01, 3.75738330e-02, 5.20508213e-04, ...,
        5.73521232e-19, 1.09207999e-17, 1.00366696e-17],
       [4.26160127e-01, 9.35166050e-03, 2.95627350e-03, ...,
        7.30360776e-18, 7.81235434e-19, 7.17401586e-18]], dtype=float32)

In [7]:
omefl2[22,:,:,1]

array([[2.15348586e-01, 3.09194606e-02, 9.46584903e-03, ...,
        1.47321198e-18, 1.12621203e-17, 3.92877621e-18],
       [4.79610683e-03, 2.07138900e-02, 4.58667614e-02, ...,
        4.03038559e-17, 1.84101933e-17, 1.03109621e-19],
       [9.81253479e-03, 3.01088579e-02, 3.81230097e-03, ...,
        1.33203776e-19, 3.30984452e-18, 4.74768502e-18],
       ...,
       [9.35612619e-02, 4.52785268e-02, 9.06258501e-05, ...,
        3.37361543e-17, 2.29487067e-17, 2.75891022e-17],
       [4.93194461e-01, 3.75738330e-02, 5.20508213e-04, ...,
        5.73521232e-19, 1.09207999e-17, 1.00366696e-17],
       [4.26160127e-01, 9.35166050e-03, 2.95627350e-03, ...,
        7.30360776e-18, 7.81235434e-19, 7.17401586e-18]], dtype=float32)

In [6]:
dfu = pd.read_csv('omega_norm.csv', header = None, dtype =np.float32)
dtest = dfu.to_numpy()
print(dtest.shape)
dtest = np.reshape(dtest, (nx,ny,nz,50), order="F")

(70875000, 1)


In [21]:
dtest[22,:,:,1]

array([[2.15348586e-01, 3.09194606e-02, 9.46584903e-03, ...,
        1.47321198e-18, 1.12621203e-17, 3.92877621e-18],
       [4.79610683e-03, 2.07138900e-02, 4.58667614e-02, ...,
        4.03038559e-17, 1.84101933e-17, 1.03109621e-19],
       [9.81253479e-03, 3.01088579e-02, 3.81230097e-03, ...,
        1.33203776e-19, 3.30984452e-18, 4.74768502e-18],
       ...,
       [9.35612619e-02, 4.52785268e-02, 9.06258501e-05, ...,
        3.37361543e-17, 2.29487067e-17, 2.75891022e-17],
       [4.93194461e-01, 3.75738330e-02, 5.20508213e-04, ...,
        5.73521232e-19, 1.09207999e-17, 1.00366696e-17],
       [4.26160127e-01, 9.35166050e-03, 2.95627350e-03, ...,
        7.30360776e-18, 7.81235434e-19, 7.17401586e-18]], dtype=float32)

In [17]:
omega[22,:,:,1]

array([[2.15348586e-01, 3.09194606e-02, 9.46584903e-03, ...,
        1.47321198e-18, 1.12621203e-17, 3.92877621e-18],
       [4.79610683e-03, 2.07138900e-02, 4.58667614e-02, ...,
        4.03038559e-17, 1.84101933e-17, 1.03109621e-19],
       [9.81253479e-03, 3.01088579e-02, 3.81230097e-03, ...,
        1.33203776e-19, 3.30984452e-18, 4.74768502e-18],
       ...,
       [9.35612619e-02, 4.52785268e-02, 9.06258501e-05, ...,
        3.37361543e-17, 2.29487067e-17, 2.75891022e-17],
       [4.93194461e-01, 3.75738330e-02, 5.20508213e-04, ...,
        5.73521232e-19, 1.09207999e-17, 1.00366696e-17],
       [4.26160127e-01, 9.35166050e-03, 2.95627350e-03, ...,
        7.30360776e-18, 7.81235434e-19, 7.17401586e-18]], dtype=float32)

In [21]:
lamb2.min()

-0.0005829274